In [1]:
import numpy as np
import pandas as pd
import pickle
import datetime

<div class="span5 alert alert-info">
<h2>Notebook details</h2>

<p> This notebook is for <b>data wranging for Machine learning</b> for <b>Mortagage customer segementation</b> project.</p>

<p> Notes.</p>
<ol>
<li>Extract all the required columns into dummy</li>
<li>Extract the salary and loan amount column in dummy range columns </li>
<li>Create new column accepted based on application accepted or denied by applicants</li>
<li> Extract data into 2 pickle files
    <ul>
     <li>File with income and loan amount as the individual columns </li>
     <li>File with income and loan amount as the range based dummy columns </li>
    </ul>
</li>        
</ol>
</div>

In [2]:
# setting some initial values and reading data from pickle file
pd.set_option('display.max_colwidth', -1)
pickle_file='df_selectdata_hmda_cenus.sa'
df_selectdata_ML = pickle.load( open( pickle_file, "rb" ) )
#df_final_months.info()
#df_final_months.head()

In [3]:
df_selectdata_ML.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 21 columns):
Year                    int64
PropertyType            int64
LoanPurpose             float64
Occupancy               float64
LoanAmount              float64
ActionType              float64
MSA                     float64
StateCode               float64
CountyCode              float64
ApplicantEthnicity      float64
CoApplicantEthnicity    float64
ApplicantRace           float64
CoApplicantRace         float64
ApplicantSex            float64
CoApplicantSex          float64
ApplicantIncome         float64
PurchaserType           float64
StateName               object
CountyName              object
CLASSCODEFIPS           object
Result                  int64
dtypes: float64(15), int64(3), object(3)
memory usage: 5.2+ GB


## Helper methods

In [4]:
# Method to convert the category column into dummy columns 
def AddDummyColumnsToDataFrame(dfinput,colname,removelastdummy=False):
    print('Add {}'.format(colname))
    temp =pd.get_dummies(dfinput[colname])
    # remove one column from dummies with least value.
  
    if removelastdummy:
        t=dfinput.groupby(colname).count().state
        col_name=((t[t.values==t.min()]).index).get_values()[0]
        if col_name in temp.columns:
            print('removed column {}'.format(col_name))
            temp=temp.drop([col_name], axis=1)
    
    # remove the main column after extracting dummy
    if colname in dfinput.columns:
        print('removed column {}'.format(colname))
        dfinput =dfinput.drop([colname], axis=1)
    for col in temp:
        temp.rename(columns={col: colname+'_'+str(col)}, inplace=True)
    
    return  pd.concat([dfinput,temp], axis=1,ignore_index=False)

In [5]:
# Method to create column value based on applicant accepted or rejected loan application
def createResultCol(row):
    if (row.ActionType==1 ):
        row.Accepted=1
    elif (row.ActionType==2 ):
        row.Accepted=0
    elif (row.ActionType==3 ):
        row.Accepted=-1
    elif (row.ActionType==4 ):
        row.Accepted=0        
    elif (row.ActionType==5 ):
        row.Accepted=np.NAN
    elif (row.ActionType==6 ):
        row.Accepted=1
    elif (row.ActionType==7 ):
        row.Accepted=-1
    elif (row.ActionType==8 ):
        row.Accepted=0        
    else:
        row.Accepted=np.NAN
    return row

In [6]:
# To reduce the size of data set and allow the code to analyze cluster in each state the data frame filtered by State
def getDataFrameforState(inputframe,stateName='CA'):
    df=inputframe[inputframe.StateName==stateName]
    df=df[df.Accepted>0]
    # create index in wrangling
    #df = df.reset_index()
    return df

### Processing Starts

####  With category starts here

In [7]:
_=df_selectdata_ML.reset_index(inplace = True)

In [8]:
df_selectdata_ML.head(5)

,index,Year,PropertyType,LoanPurpose,Occupancy,LoanAmount,ActionType,MSA,StateCode,CountyCode,...,ApplicantRace,CoApplicantRace,ApplicantSex,CoApplicantSex,ApplicantIncome,PurchaserType,StateName,CountyName,CLASSCODEFIPS,Result
0,0,2015,1,3.0,1.0,222.0,3.0,11260.0,2.0,170.0,...,5.0,8.0,1.0,5.0,115.0,0.0,AK,Matanuska-Susitna Borough,H1,0
1,1,2015,1,3.0,2.0,188.0,3.0,11260.0,2.0,170.0,...,5.0,8.0,1.0,5.0,115.0,0.0,AK,Matanuska-Susitna Borough,H1,0
2,2,2015,1,3.0,2.0,216.0,1.0,11260.0,2.0,20.0,...,5.0,8.0,1.0,5.0,230.0,7.0,AK,Anchorage Municipality,H6,1
3,3,2015,1,3.0,1.0,219.0,1.0,11260.0,2.0,170.0,...,5.0,8.0,1.0,5.0,145.0,1.0,AK,Matanuska-Susitna Borough,H1,1
4,4,2015,1,3.0,2.0,188.0,1.0,11260.0,2.0,170.0,...,5.0,8.0,1.0,5.0,145.0,1.0,AK,Matanuska-Susitna Borough,H1,1


In [9]:
# filter out suspened, cancelled and live projects
#Type of Action Taken* (1,6)** Yes (2,8,4) ** No (3,7)**denied (5)*****NA
print(str(datetime.datetime.now()))
df_selectdata_ML['Accepted']=0
df_selectdata_ML=df_selectdata_ML.apply(createResultCol,axis=1)
print(str(datetime.datetime.now()))

2018-07-08 10:14:26.350261
2018-07-08 11:29:26.113986


In [10]:
# Convert category columns to dummy columns=
#df_selectdata_ML=df_selectdata
categoryColumns=['StateCode','ApplicantEthnicity','ApplicantRace','ApplicantSex','Occupancy','PropertyType','LoanPurpose']
for col in categoryColumns:
    print(col)
    df_selectdata_ML=AddDummyColumnsToDataFrame(df_selectdata_ML,col)

StateCode
Add StateCode
removed column StateCode
ApplicantEthnicity
Add ApplicantEthnicity
removed column ApplicantEthnicity
ApplicantRace
Add ApplicantRace
removed column ApplicantRace
ApplicantSex
Add ApplicantSex
removed column ApplicantSex
Occupancy
Add Occupancy
removed column Occupancy
PropertyType
Add PropertyType
removed column PropertyType
LoanPurpose
Add LoanPurpose
removed column LoanPurpose


In [11]:
# Add column for loan amount
df_selectdata_ML=df_selectdata_ML.assign(SAL_0_50=((df_selectdata_ML['ApplicantIncome']>0) & (df_selectdata_ML['ApplicantIncome']<=50)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_50_100=((df_selectdata_ML['ApplicantIncome']>50) & (df_selectdata_ML['ApplicantIncome']<=100)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_100_150=((df_selectdata_ML['ApplicantIncome']>100) & (df_selectdata_ML['ApplicantIncome']<=150)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_150_200=((df_selectdata_ML['ApplicantIncome']>150) & (df_selectdata_ML['ApplicantIncome']<=200)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_200_250=((df_selectdata_ML['ApplicantIncome']>200) & (df_selectdata_ML['ApplicantIncome']<=250)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_250_300=((df_selectdata_ML['ApplicantIncome']>250) & (df_selectdata_ML['ApplicantIncome']<=300)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_300_350=((df_selectdata_ML['ApplicantIncome']>300) & (df_selectdata_ML['ApplicantIncome']<=350)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_350_400=((df_selectdata_ML['ApplicantIncome']>350) & (df_selectdata_ML['ApplicantIncome']<=400)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_400_450=((df_selectdata_ML['ApplicantIncome']>400) & (df_selectdata_ML['ApplicantIncome']<=450)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_450_500=((df_selectdata_ML['ApplicantIncome']>450) & (df_selectdata_ML['ApplicantIncome']<=500)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_500_5500=((df_selectdata_ML['ApplicantIncome']>500) & (df_selectdata_ML['ApplicantIncome']<=5500)))
df_selectdata_ML=df_selectdata_ML.assign(SAL_5500_999999=((df_selectdata_ML['ApplicantIncome']>5500) & (df_selectdata_ML['ApplicantIncome']<=999999)))

In [12]:
df_selectdata_ML['SAL_0_50'] = (df_selectdata_ML['SAL_0_50'] == True).astype(int)
df_selectdata_ML['SAL_50_100'] = (df_selectdata_ML['SAL_50_100'] == True).astype(int)
df_selectdata_ML['SAL_100_150'] = (df_selectdata_ML['SAL_100_150'] == True).astype(int)
df_selectdata_ML['SAL_150_200'] = (df_selectdata_ML['SAL_150_200'] == True).astype(int)
df_selectdata_ML['SAL_200_250'] = (df_selectdata_ML['SAL_200_250'] == True).astype(int)
df_selectdata_ML['SAL_250_300'] = (df_selectdata_ML['SAL_250_300'] == True).astype(int)
df_selectdata_ML['SAL_300_350'] = (df_selectdata_ML['SAL_300_350'] == True).astype(int)
df_selectdata_ML['SAL_350_400'] = (df_selectdata_ML['SAL_350_400'] == True).astype(int)
df_selectdata_ML['SAL_400_450'] = (df_selectdata_ML['SAL_400_450'] == True).astype(int)
df_selectdata_ML['SAL_450_500'] = (df_selectdata_ML['SAL_450_500'] == True).astype(int)
df_selectdata_ML['SAL_500_5500'] = (df_selectdata_ML['SAL_500_5500'] == True).astype(int)
df_selectdata_ML['SAL_5500_999999'] = (df_selectdata_ML['SAL_5500_999999'] == True).astype(int)

In [13]:
# Add column for loan amount
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_0_50=((df_selectdata_ML['LoanAmount']>0) & (df_selectdata_ML['LoanAmount']<=50)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_50_100=((df_selectdata_ML['LoanAmount']>50) & (df_selectdata_ML['LoanAmount']<=100)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_100_150=((df_selectdata_ML['LoanAmount']>100) & (df_selectdata_ML['LoanAmount']<=150)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_150_200=((df_selectdata_ML['LoanAmount']>150) & (df_selectdata_ML['LoanAmount']<=200)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_200_250=((df_selectdata_ML['LoanAmount']>200) & (df_selectdata_ML['LoanAmount']<=250)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_250_300=((df_selectdata_ML['LoanAmount']>250) & (df_selectdata_ML['LoanAmount']<=300)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_300_350=((df_selectdata_ML['LoanAmount']>300) & (df_selectdata_ML['LoanAmount']<=350)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_350_400=((df_selectdata_ML['LoanAmount']>350) & (df_selectdata_ML['LoanAmount']<=400)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_400_450=((df_selectdata_ML['LoanAmount']>400) & (df_selectdata_ML['LoanAmount']<=450)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_450_500=((df_selectdata_ML['LoanAmount']>450) & (df_selectdata_ML['LoanAmount']<=500)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_500_5500=((df_selectdata_ML['LoanAmount']>500) & (df_selectdata_ML['LoanAmount']<=5500)))
df_selectdata_ML=df_selectdata_ML.assign(LonAmt_5500_999999=((df_selectdata_ML['LoanAmount']>5500) & (df_selectdata_ML['LoanAmount']<=999999)))

In [14]:
df_selectdata_ML['LonAmt_0_50'] = (df_selectdata_ML['LonAmt_0_50'] == True).astype(int)
df_selectdata_ML['LonAmt_50_100'] = (df_selectdata_ML['LonAmt_50_100'] == True).astype(int)
df_selectdata_ML['LonAmt_100_150'] = (df_selectdata_ML['LonAmt_100_150'] == True).astype(int)
df_selectdata_ML['LonAmt_150_200'] = (df_selectdata_ML['LonAmt_150_200'] == True).astype(int)
df_selectdata_ML['LonAmt_200_250'] = (df_selectdata_ML['LonAmt_200_250'] == True).astype(int)
df_selectdata_ML['LonAmt_250_300'] = (df_selectdata_ML['LonAmt_250_300'] == True).astype(int)
df_selectdata_ML['LonAmt_300_350'] = (df_selectdata_ML['LonAmt_300_350'] == True).astype(int)
df_selectdata_ML['LonAmt_350_400'] = (df_selectdata_ML['LonAmt_350_400'] == True).astype(int)
df_selectdata_ML['LonAmt_400_450'] = (df_selectdata_ML['LonAmt_400_450'] == True).astype(int)
df_selectdata_ML['LonAmt_450_500'] = (df_selectdata_ML['LonAmt_450_500'] == True).astype(int)
df_selectdata_ML['LonAmt_500_5500'] = (df_selectdata_ML['LonAmt_500_5500'] == True).astype(int)
df_selectdata_ML['LonAmt_5500_999999'] = (df_selectdata_ML['LonAmt_5500_999999'] == True).astype(int)

In [15]:
df_selectdata_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31630026 entries, 0 to 31630025
Columns: 116 entries, index to LonAmt_5500_999999
dtypes: float64(9), int32(24), int64(4), object(3), uint8(76)
memory usage: 8.8+ GB


In [17]:
#df_selecteddata.info()
picklefilename='pickle_selectdata_ML_All_Col.sa'
# get the pickle file name to re confirm the previous files are not overriden.
# Note: Run this file after confimring succesful completion of all above steps
print(picklefilename)
# create pickle file for further use 
pickle.dump(df_selectdata_ML,open(picklefilename,'wb'), protocol=4)

pickle_selectdata_ML_All_Col.sa


In [18]:
# Get only for CA only
df_selectdata_ML_sel_CA=getDataFrameforState(df_selectdata_ML,'CA')

In [19]:
#df_selecteddata.info()
picklefilename='pickle_selectdata_ML_All_Col_CA.sa'
# get the pickle file name to re confirm the previous files are not overriden.
# Note: Run this file after confimring succesful completion of all above steps
print(picklefilename)
# create pickle file for further use 
pickle.dump(df_selectdata_ML_sel_CA,open(picklefilename,'wb'), protocol=4)

pickle_selectdata_ML_All_Col_CA.sa


In [ ]:
df_selectdata_ML_sel=df_selectdata_ML_sel.loc[:,['Year',
'CountyCode',
'PurchaserType',
'StateName',
'CountyName',
'Result',
'Accepted',
'StateCode_1.0',
'StateCode_2.0',
'StateCode_4.0',
'StateCode_5.0',
'StateCode_6.0',
'StateCode_8.0',
'StateCode_9.0',
'StateCode_10.0',
'StateCode_11.0',
'StateCode_12.0',
'StateCode_13.0',
'StateCode_15.0',
'StateCode_16.0',
'StateCode_17.0',
'StateCode_18.0',
'StateCode_19.0',
'StateCode_20.0',
'StateCode_21.0',
'StateCode_22.0',
'StateCode_23.0',
'StateCode_24.0',
'StateCode_25.0',
'StateCode_26.0',
'StateCode_27.0',
'StateCode_28.0',
'StateCode_29.0',
'StateCode_30.0',
'StateCode_31.0',
'StateCode_32.0',
'StateCode_33.0',
'StateCode_34.0',
'StateCode_35.0',
'StateCode_36.0',
'StateCode_37.0',
'StateCode_38.0',
'StateCode_39.0',
'StateCode_40.0',
'StateCode_41.0',
'StateCode_42.0',
'StateCode_44.0',
'StateCode_45.0',
'StateCode_46.0',
'StateCode_47.0',
'StateCode_48.0',
'StateCode_49.0',
'StateCode_50.0',
'StateCode_51.0',
'StateCode_53.0',
'StateCode_54.0',
'StateCode_55.0',
'StateCode_56.0',
'StateCode_72.0',
'StateCode_78.0',
'ApplicantEthnicity_1.0',
'ApplicantEthnicity_2.0',
'ApplicantEthnicity_3.0',
'ApplicantEthnicity_4.0',
'ApplicantRace_1.0',
'ApplicantRace_2.0',
'ApplicantRace_3.0',
'ApplicantRace_4.0',
'ApplicantRace_5.0',
'ApplicantRace_6.0',
'ApplicantRace_7.0',
'ApplicantSex_1.0',
'ApplicantSex_2.0',
'ApplicantSex_3.0',
'ApplicantSex_4.0',
'Occupancy_1.0',
'Occupancy_2.0',
'Occupancy_3.0',
'PropertyType_1',
'PropertyType_2',
'LoanPurpose_1.0',
'LoanPurpose_2.0',
'LoanPurpose_3.0',  
'SAL_0_50',
'SAL_50_100',
'SAL_100_150',
'SAL_150_200',
'SAL_200_250',
'SAL_250_300',
'SAL_300_350',
'SAL_350_400',
'SAL_400_450',
'SAL_450_500',
'SAL_500_5500',
'SAL_5500_999999',
'LonAmt_0_50',
'LonAmt_50_100',
'LonAmt_100_150',
'LonAmt_150_200',
'LonAmt_200_250',
'LonAmt_250_300',
'LonAmt_300_350',
'LonAmt_350_400',
'LonAmt_400_450',
'LonAmt_450_500',
'LonAmt_500_5500',
'LonAmt_5500_999999']]

In [ ]:
for col in df_selectdata_ML_sel:
    print("'"+col+"',")

### Dataframe with salary and loan amount as columns

In [ ]:
df_selectdata_ML_sel=df_selectdata_ML

In [ ]:
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('CoApplicantEthnicity',axis=1)
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('CoApplicantRace',axis=1)
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('CoApplicantSex',axis=1)
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('CLASSCODEFIPS',axis=1)
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('ActionType',axis=1)
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('MSA',axis=1)

In [ ]:
df_selectdata_ML_sel=df_selectdata_ML_sel.loc[:,['Year',
'CountyCode',
'PurchaserType',
'StateName',
'CountyName',
'Result',
'Accepted',
'StateCode_1.0',
'StateCode_2.0',
'StateCode_4.0',
'StateCode_5.0',
'StateCode_6.0',
'StateCode_8.0',
'StateCode_9.0',
'StateCode_10.0',
'StateCode_11.0',
'StateCode_12.0',
'StateCode_13.0',
'StateCode_15.0',
'StateCode_16.0',
'StateCode_17.0',
'StateCode_18.0',
'StateCode_19.0',
'StateCode_20.0',
'StateCode_21.0',
'StateCode_22.0',
'StateCode_23.0',
'StateCode_24.0',
'StateCode_25.0',
'StateCode_26.0',
'StateCode_27.0',
'StateCode_28.0',
'StateCode_29.0',
'StateCode_30.0',
'StateCode_31.0',
'StateCode_32.0',
'StateCode_33.0',
'StateCode_34.0',
'StateCode_35.0',
'StateCode_36.0',
'StateCode_37.0',
'StateCode_38.0',
'StateCode_39.0',
'StateCode_40.0',
'StateCode_41.0',
'StateCode_42.0',
'StateCode_44.0',
'StateCode_45.0',
'StateCode_46.0',
'StateCode_47.0',
'StateCode_48.0',
'StateCode_49.0',
'StateCode_50.0',
'StateCode_51.0',
'StateCode_53.0',
'StateCode_54.0',
'StateCode_55.0',
'StateCode_56.0',
'StateCode_72.0',
'StateCode_78.0',
'ApplicantEthnicity_1.0',
'ApplicantEthnicity_2.0',
'ApplicantEthnicity_3.0',
'ApplicantEthnicity_4.0',
'ApplicantRace_1.0',
'ApplicantRace_2.0',
'ApplicantRace_3.0',
'ApplicantRace_4.0',
'ApplicantRace_5.0',
'ApplicantRace_6.0',
'ApplicantRace_7.0',
'ApplicantSex_1.0',
'ApplicantSex_2.0',
'ApplicantSex_3.0',
'ApplicantSex_4.0',
'Occupancy_1.0',
'Occupancy_2.0',
'Occupancy_3.0',
'PropertyType_1',
'PropertyType_2',
'LoanPurpose_1.0',
'LoanPurpose_2.0',
'LoanPurpose_3.0', 
'ApplicantIncome',
'LoanAmount']]

In [ ]:
#df_selecteddata.info()
picklefilename='df_selectdata_ML_WSA.sa'
# get the pickle file name to re confirm the previous files are not overriden.
# Note: Run this file after confimring succesful completion of all above steps
print(picklefilename)
# create pickle file for further use 
pickle.dump(df_selectdata_ML_sel,open(picklefilename,'wb'), protocol=4)

In [ ]:
len(df_selectdata_ML_sel)

#### Start without dummy columns

In [ ]:
#df_selectdata_ML_sel=df_selectdata_ML#.iloc[:,13:]
df_selectdata_ML_sel=df_selectdata#.iloc[:,13:]

In [ ]:
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('CoApplicantEthnicity',axis=1)
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('CoApplicantRace',axis=1)
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('CoApplicantSex',axis=1)
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('CLASSCODEFIPS',axis=1)
#df_selectdata_ML_sel=df_selectdata_ML_sel.drop('ActionType',axis=1)
df_selectdata_ML_sel=df_selectdata_ML_sel.drop('MSA',axis=1)

In [ ]:
df_selectdata_ML_sel=df_selectdata_ML_sel.loc[:,['Year',
'PurchaserType',
'CountyCode',
'StateName',
'CountyName',
'StateCode',
'Result',
'Accepted',
'ApplicantEthnicity',
'ApplicantRace',
'ApplicantSex',
'ApplicantIncome', 
'Occupancy',
'PropertyType',
'LoanPurpose',
'LoanAmount',
'ActionType'
]]

In [ ]:
for col in df_selectdata_ML_sel:
    print("'"+col+"',")

In [ ]:
#df_selecteddata.info()
picklefilename='df_selectdata_W_Col.sa'
# get the pickle file name to re confirm the previous files are not overriden.
# Note: Run this file after confimring succesful completion of all above steps
print(picklefilename)
# create pickle file for further use 
pickle.dump(df_selectdata_ML_sel,open(picklefilename,'wb'), protocol=4)

In [ ]:
#rough

In [ ]:
%reset